# Package ingest

In [62]:
# Importing Python packages

import pandas as pd
import numpy as np
import plotnine as p9
import os
import geopandas as gpd
from pathlib import Path


# Data ingest

In [63]:
# Reading in phased development data
phased_development = pd.read_csv('pipeline_example_output.csv')
phased_development.head(1)

,Unnamed: 0,ProjectID,land_use,year,percent_development_pa,status,development_pa
0,0,2100.0,Residential - Villas Emiratis (Units),2024,0.5,Under Construction,0.0


In [64]:
# Reading in baseline TAZ data

sharepoint_path = f'C:/Users/{os.environ.get( "USERNAME" )}/Buro Happold'
project_path = Path(f"{sharepoint_path}/P060927 Plan Capital 2040 - 02_Documents/11_GIS/05 Urban Models")
ium_path = os.path.join(project_path,'ium_ad.gpkg')
zone_taz_2022 = gpd.read_file(ium_path, layer='zone_taz_2022')
zone_taz_2022.head(1)

,Z,SHAPE_Length,SHAPE_Area,NUM_OF_BUILDINGS,Calc_Total_Ex_GFA,Ex_GFA,RES_GFA,Residential_Labour_GFA,OFFICE_GFA,RETAIL_GFA,...,RETMAL_PRO,MEDCLI_PRO,MEDHOS_PRO,NONTER_PRO,TER_PRO,SCHPLCS_NT,SCHPLCS_TE,SCHPLCS_PR,SCHPLCS_SE,geometry
0,1044,46364.0,600150.0,675.0,295592.0,296118.0,269986.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((270352.336 2727199.707, 269735..."


In [65]:
# Bringing in MP to TAZ development deltas

zone_taz_delta = gpd.read_file(ium_path, layer='zone_taz_delta')
zone_taz_delta.head(1)


,Z,ZONE,EPA,UPC_PRECIN,PRECINCT_N,ST_DIST_ID,ST_DIST_EN,UDM_DIST_I,UDM_DIST_E,REGION,...,2022_SCHPLCS_PR,2022_SCHPLCS_SE,masterplan_projectid,masterplan_overlap_proportion,masterplan_overlap_area_m2,tod_intersection_area,tod_intersection_pct,protected_intersection_area,protected_intersection_pct,geometry
0,1044,1044,None,7.0,Al Rahba,88.0,Al Rahba,1.0,AL RAHBA,ADM,...,None,None,None,NaN,NaN,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((270352.336 2727199.707, 269735..."


In [66]:
zone_taz_delta

Index(['Z', 'ZONE', 'EPA', 'UPC_PRECIN', 'PRECINCT_N', 'ST_DIST_ID',
       'ST_DIST_EN', 'UDM_DIST_I', 'UDM_DIST_E', 'REGION', 'AREA', 'CENTROIDX',
       'CENTROIDY', 'TMP_NAME', 'AREATYPE', '2022_Z', '2022_SHAPE_Length',
       '2022_SHAPE_Area', '2022_NUM_OF_BUILDINGS', '2022_Calc_Total_Ex_GFA',
       '2022_Ex_GFA', '2022_RES_GFA', '2022_Residential_Labour_GFA',
       '2022_OFFICE_GFA', '2022_RETAIL_GFA', '2022_Hotel_GFA',
       '2022_Commercial_Recreation_GFA', '2022_Industrial_GFA',
       '2022_SCHOOL_GFA', '2022_MED_GFA', '2022_RELIGIOUS_GFA',
       '2022_GOV_INST_SOC_CULT_GFA', '2022_OPENSPACE_RECREATION_GFA',
       '2022_Community_Facilities_GFA', '2022_AGRICULTURE_GFA',
       '2022_OTHER_GFA', '2022_ST_Other_GFA', '2022_INFRASTRUCTURE_GFA',
       '2022_Hotel_Rooms', '2022_Villa_Units', '2022_Apt_Units',
       '2022_DC_Max_GFA', '2022_DC_Res_GFA', '2022_DC_Office_GFA',
       '2022_DC_Retail_GFA', '2022_DC_Hotel_GFA', '2022_DC_Industrial_GFA',
       '2022_Allowed_GFA

In [67]:
# Ingesting MP land use to TAZ land use crosswalk

crosswalks = pd.read_csv('taxonomy_crosswalks.csv')
crosswalks = crosswalks[crosswalks['crosswalk'] == 'MP Committed Development to GIS Use']

# Filtering out the following land uses from the crosswalk dataframe

mp_land_use_exclusion = ['Residential - Labour Accommodation (Units)'] #  Land uses to exclude for the crosswalk
crosswalks = crosswalks[~crosswalks['variable_1'].isin(mp_land_use_exclusion)] # Filtering the crosswalk
crosswalks = crosswalks.drop('Notes',axis = 1) # Dropping notes column
crosswalks.head(3)

,crosswalk,variable_1,variable_2
0,MP Committed Development to GIS Use,Residential - Villas Emiratis (Units),Villa_Units
1,MP Committed Development to GIS Use,Residential - Villas Non-Emiratis (Units),Villa_Units
2,MP Committed Development to GIS Use,Residential - Apartments (Units),Apt_Units


In [68]:
# Trimming down crosswalk

crosswalks


,crosswalk,variable_1,variable_2
0,MP Committed Development to GIS Use,Residential - Villas Emiratis (Units),Villa_Units
1,MP Committed Development to GIS Use,Residential - Villas Non-Emiratis (Units),Villa_Units
2,MP Committed Development to GIS Use,Residential - Apartments (Units),Apt_Units
4,MP Committed Development to GIS Use,Retail (GFA),RETAIL_GFA
5,MP Committed Development to GIS Use,Retail (GFA),Commercial_Recreation_GFA
6,MP Committed Development to GIS Use,Office (GFA),OFFICE_GFA
7,MP Committed Development to GIS Use,Industrial (GFA),INDUSTRIAL_GFA
8,MP Committed Development to GIS Use,Hotel (Units),Hotel_Rooms
9,MP Committed Development to GIS Use,Community Facilities (GFA),RELIGIOUS_GFA
10,MP Committed Development to GIS Use,Community Facilities (GFA),GOV_INST_SOC_CULT_GFA


In [69]:
phased_development

,Unnamed: 0,ProjectID,land_use,year,percent_development_pa,status,development_pa
0,0,2100.0,Residential - Villas Emiratis (Units),2024,0.50,Under Construction,0.00
1,1,5000.0,Residential - Villas Emiratis (Units),2024,0.50,Under Construction,0.00
2,2,1996.0,Residential - Villas Emiratis (Units),2024,0.50,Under Construction,710.00
3,3,1995.0,Residential - Villas Emiratis (Units),2024,0.50,Under Construction,0.00
4,4,ICAD 1,Residential - Villas Emiratis (Units),2024,0.50,Under Construction,0.00
...,...,...,...,...,...,...,...
33043,33043,Riyadh,Hotel (Units),2040,0.05,Override,15.30
33044,33044,Riyadh,Community Facilities (GFA),2040,0.05,Override,3264.85
33045,33045,Riyadh,Healthcare (GFA),2040,0.05,Override,0.00
33046,33046,Riyadh,Education (GFA),2040,0.05,Override,0.00
